This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'rrqSvs1Qbb9xebiNoM-f'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [ ]:
# First, import the relevant modules
import requests
import json

In [ ]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

payload = {'start_date': '2020-11-02', 'end_date': '2020-11-03'}
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=' + API_KEY, params=payload)

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.text

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [1]:
import requests
import json

#GET TIME SERIES INTO A DICTIONARY
API_KEY = 'rrqSvs1Qbb9xebiNoM-f'

payload = {'start_date': '2017-01-01', 'end_date': '2017-12-31'}

r = requests.get(
        'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=' 
        + API_KEY, params=payload)

AFX = json.loads(r.text)

#INDEX DICTIONARY TO BE USED TO CALL DATA FROM DICTIONARY
idx = dict(zip(
        AFX['dataset_data']['column_names'],
        [i for i in range(0,len(AFX['dataset_data']['column_names']))] )) 

#INITIALIZING LOOP VARIABLES
max_open = 0
min_open = 10000
max_oneday_delta = 0
max_twoday_delta = 0
trade_volume = []
last_closing_price = AFX['dataset_data']['data'][0][idx['Close']]

#LOOPING THROUGH THE DATA
for row in AFX['dataset_data']['data']: 
    if row[idx['Open']] != None:
        max_open = max(max_open,row[idx['Open']])
        min_open = min(min_open,row[idx['Open']])
        
    if (row[idx['Low']] != None)*(row[idx['Low']] != None):
        max_oneday_delta = max(
                            max_oneday_delta,
                            row[idx['High']] - row[idx['Low']])
        
    if row[idx['Traded Volume']] != None:
        trade_volume.append(row[idx['Traded Volume']])
        
    max_twoday_delta = max(
                        max_twoday_delta,
                        abs(row[idx['Close']] - last_closing_price))
    
    last_closing_price = row[idx['Close']]

#COMPUTING DESCRIPTIVE STATICTICS ON TRADE VOLUME

mean_tvol = sum(trade_volume) / len(trade_volume)

trade_volume.sort()

median_tvol = trade_volume[int(len(trade_volume) / 2)]

print('Highest Open: ' + str(max_open))
print('Lowest Open: ' + str(min_open))
print('Largest Day Change: ' + str(round(max_oneday_delta,2)))
print('Largest 2-Day Change: ' + str(round(max_twoday_delta,2)))
print('Average daily Trading Volume: ' + str(round(mean_tvol,2)))
print('Median daily Trading Volume: ' + str(round(median_tvol,2)))

Highest Open: 53.11
Lowest Open: 34.0
Largest Day Change: 2.81
Largest 2-Day Change: 2.56
Average daily Trading Volume: 89124.34
Median daily Trading Volume: 76286.0
